In [11]:
import nltk
import math
import pickle
import re
import time
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
inputFileName = "data.tsv"   
testFileName = "eval1_unlabelled.tsv"  
outputFileName = "answer.tsv"

!pip install PyDrive

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '19ml1naPkyeVp_xTi5NXAqvEmAO-yfE-1'})
download.GetContentFile(inputFileName)

download = drive.CreateFile({'id': '1-G2XRn85mYTwjUaiYGUgD9_pHZh96qCU'})
download.GetContentFile(testFileName)


    100% |████████████████████████████████| 993kB 20.4MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
docIDFDict = {}
avgDocLength = 0

docFrequencyDict = {}   
numOfDocuments = 0   
totalDocLength = 0
table = str.maketrans('', '', string.punctuation)
stop_words = stopwords.words('english')
porter = PorterStemmer()

for line in open(inputFileName,"r"):
	passage = line.strip().lower().split("\t")[2]

	doc = re.split('\W+', passage)
	doc = [x.lower() for x in doc if x and not x in stop_words] # to remove empty doc 
	doc = [w.translate(table) for w in doc]
	doc = [porter.stem(word) for word in doc]

	totalDocLength += len(doc)

	doc = list(set(doc))

	for word in doc : #Updates n(q_i) values for all the words(q_i)
		if word not in docFrequencyDict :
			docFrequencyDict[word] = 0
		docFrequencyDict[word] += 1
	   
	numOfDocuments = numOfDocuments + 1
	if (numOfDocuments%5000==0):
		print(numOfDocuments)  

for word in docFrequencyDict:  #Calculate IDF scores for each word(q_i)
	docIDFDict[word] = math.log((numOfDocuments - docFrequencyDict[word] +0.5) / (docFrequencyDict[word] + 0.5), math.e) #Why are you considering "numOfDocuments - docFrequencyDict[word]" instead of just "numOfDocuments"

avgDocLength = totalDocLength / numOfDocuments

     
pickle_out = open("docIDFDict.pickle","wb") # Saves IDF scores in pickle file, which is optional
pickle.dump(docIDFDict, pickle_out)
pickle_out.close()


print("NumOfDocuments : ", numOfDocuments)
print("AvgDocLength : ", avgDocLength)

In [0]:
docIDFDict = {}
avgDocLength = 0

docFrequencyDict = {}   
numOfDocuments = 0   
totalDocLength = 0
table = str.maketrans('', '', string.punctuation)
stop_words = {}
for word in stopwords.words('english'):
	stop_words[word] = 1


porter = PorterStemmer()

for line in open(inputFileName,"r"):
	passage = line.strip().lower().split("\t")[2]

	doc = re.split('\W+', passage)
	doc = [porter.stem(x.lower().translate(table)) for x in doc if x and stop_words.get(x,0)!=1] 

	totalDocLength += len(doc)

	doc = list(set(doc))

	for word in doc : #Updates n(q_i) values for all the words(q_i)
		if word not in docFrequencyDict :
			docFrequencyDict[word] = 0
		docFrequencyDict[word] += 1
	   
	numOfDocuments = numOfDocuments + 1
	if (numOfDocuments%10000==0):
		print(numOfDocuments)  

for word in docFrequencyDict:  #Calculate IDF scores for each word(q_i)
	docIDFDict[word] = math.log((numOfDocuments - docFrequencyDict[word] +0.5) / (docFrequencyDict[word] + 0.5), math.e) #Why are you considering "numOfDocuments - docFrequencyDict[word]" instead of just "numOfDocuments"

avgDocLength = totalDocLength / numOfDocuments

     
pickle_out = open("docIDFDict.pickle","wb") # Saves IDF scores in pickle file, which is optional
pickle.dump(docIDFDict, pickle_out)
pickle_out.close()


print("NumOfDocuments : ", numOfDocuments)
print("AvgDocLength : ", avgDocLength)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
73000

In [0]:
def GetBM25Score(Query, Passage, k1=1.5, b=0.75, delimiter=' ') :
    
    global docIDFDict,avgDocLength

    query_words= Query.strip().lower().split(delimiter)
    passage_words = Passage.strip().lower().split(delimiter)
    passageLen = len(passage_words)
    docTF = {}
    for word in set(query_words):   #Find Term Frequency of all query unique words
        docTF[word] = passage_words.count(word)
    commonWords = set(query_words) & set(passage_words)
    tmp_score = []
    for word in commonWords :   
        numer = (docTF[word] * (k1+1))   #Numerator part of BM25 Formula
        denom = ((docTF[word]) + k1*(1 - b + b*passageLen/avgDocLength)) #Denominator part of BM25 Formula 
        if(word in docIDFDict) :
            tmp_score.append(docIDFDict[word] * numer / denom)

    score = sum(tmp_score)
    return score


In [0]:
def RunBM25OnEvaluationSet(testfile,outputfile):

    lno=0
    tempscores=[]  #This will store scores of 10 query,passage pairs as they belong to same query
    f = open(testfile,"r")
    fw = open(outputfile,"w")
    for line in f:
        tokens = line.strip().lower().split("\t")
        Query = tokens[1]
        Passage = tokens[2]
        score = GetBM25Score(Query,Passage) 
        tempscores.append(score)
        lno+=1
        if(lno%10==0):
            tempscores = [str(s) for s in tempscores]
            scoreString = "\t".join(tempscores)
            qid = tokens[0]
            fw.write(qid+"\t"+scoreString+"\n")
            tempscores=[]
        if(lno%5000==0):
            print(lno)
    print(lno)
    f.close()
    fw.close()

In [0]:
RunBM25OnEvaluationSet(testFileName,outputFileName)